In [ ]:
#!pip install datasets
#!pip install tokenizers==0.10.0rc1
#!pip install wandb

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

import datasets
import tokenizers
import wandb
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [3]:
text_dataset = datasets.load_dataset("imdb")

**IMDB** - это датасет по классификации эмоциональной окраски. Вам нужно предсказать положительный ли отзыв к фильму по его тексту. Это довольно простая задача и она хорошо решается даже линейными моделями. Для доступа к нему мы используем библиотеку `datasets` - она содержит в себе много интересных текстовых датасетов.

Тренировочная и тстовая части IMDB достаточно большие - каждая состоит из 25 тысяч примеров.

In [3]:
text_dataset

Как мы видим, классы сбалансированны, что позволяет использовать accuracy как простую и интерпретируемую метрику, хорошо показывающую качество модели.

In [4]:
train_labels = [e['label'] for e in text_dataset['train']]
plt.hist(train_labels)
plt.show()

In [5]:
text_dataset['train']

In [6]:
text_dataset['train']['text'][0]

# Classification using a linear model

Линейная модель - очень сильный бейзлайн и в некоторых задачах классификации вам даже не надо идти дальше линейной модели - она уже достаточно хороша. А также её можно написать менее чем в 10 строчекю Поэтому всегда стоит начинать решение любой задачи с ленейного бейзлайна.

Давайте вспомним библиотеку `sklearn` и напишем линейную модельку. Для векторизации текста мы будем использовать `TfidfVectorizer`, а в качестве модели `LogisticRegression`.


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
# TASK 1.1: create TfidfVectorizer object and fit it on out training set texts
# Our implementation is 2 lines
# YOUR CODE STARTS
vectorizer = TfidfVectorizer()
vectorizer.fit(text_dataset['train']['text'])

# YOUR CODE ENDS

In [9]:
# TASK 1.2:
# 1. convert your texts to tf-idf vectors using .transform (training texts and test texts too)
# 2. convert your labels into numpy arrays (both training and test labels)
# Or implementatin is 4 lines

# YOUR CODE STARTS
X_train = vectorizer.transform(text_dataset['train']['text']).toarray()
y_train = text_dataset['train']['label']

X_test = vectorizer.transform(text_dataset['test']['text']).toarray()
y_test = text_dataset['test']['label']
# YOUR CODE ENDS

In [10]:
X_train, y_train

In [11]:
# TASK 1.3: create LogisticRegression model object and fit the model
# Our implementation is 2 lines
# YOUR CODE STARTS

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)


# YOUR CODE ENDS

А теперь мы используем нашу модель для того, чтобы предсказать классы на тестовом сете и считаем accuracy.

In [12]:
predictions = model.predict(X_test)

print(type(predictions))
print(type(y_test))

In [13]:
# note that we can use vector operations, because we deal with numpy tensors
accuracy = (predictions == y_test).mean()
accuracy

**OMG so accurate, much machine learning**

Давайте предскажем позитивны ли такие комментарии:

In [14]:
positive_comment = 'This movie is awesome!'

vec = vectorizer.transform([positive_comment])
model.predict(vec)

In [15]:
negative_comment = 'This movie is awful!'

vec = vectorizer.transform([negative_comment])
model.predict(vec)

Как мы увидели, даже такая простая модель может хорошо классифицировать текст (accuracy 0.9 - это довольно много, тк выборка сбалансированна).

Кстати, использование модели LinearSVM обычно работает даже лучше, чем логистическая регрессия. Рекомендуем попробовать и сравнить.

Что такое SVM: [тык](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47)

Ещё один простой метод улучшить линейную модель - использовать n-gram в вашем TF-IDF. Не забудьте указать параметр `max_features` (хорошое число 50 000), а то при большом количестве фичей модель может начать переобучаться.

## LinearSVM

In [17]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(random_state=0).fit(X_train, y_train)


In [18]:
predictions = svm_model.predict(X_test)
accuracy = (predictions == y_test).mean()
accuracy

In [19]:
positive_comment = 'This movie is awesome!'

vec = vectorizer.transform([positive_comment])
model.predict(vec)

In [20]:
negative_comment = 'This movie is awful!'

vec = vectorizer.transform([negative_comment])
model.predict(vec)

## ngrams

In [5]:
vectorizer_n = TfidfVectorizer(ngram_range=(1,2), max_features=50000)
vectorizer_n.fit(text_dataset['train']['text'])

In [6]:
X_train = vectorizer_n.transform(text_dataset['train']['text']).toarray()
y_train = text_dataset['train']['label']

X_test = vectorizer_n.transform(text_dataset['test']['text']).toarray()
y_test = text_dataset['test']['label']

In [8]:
len(X_train[0])

In [9]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(random_state=0).fit(X_train, y_train)

In [10]:
predictions = svm_model.predict(X_test)
accuracy = (predictions == y_test).mean()
accuracy

In [13]:
positive_comment = 'This movie is awesome!'

vec = vectorizer_n.transform([positive_comment])
svm_model.predict(vec)

In [14]:
negative_comment = 'This movie is awful!'

vec = vectorizer_n.transform([negative_comment])
svm_model.predict(vec)